<img src="img/header1.png"></img>

<h1 style="text-align: left;">ToT: PySpark Basics to Inter./Adv. in ± 15 Minutes</h1>
By: Imam Cholissodin | Dosen Fakultas Ilmu Komputer (FILKOM), Universitas Brawijaya (UB)

# **getTweetStreaming**: getTweet by TCP Socket Stream

In [1]:
# install dan import library tweep
#!pip install tweepy

In [ ]:
from tweepy import OAuthHandler, API, Stream
from tweepy.streaming import StreamListener
import socket

tracks = ['covid-19','indonesia','jokowi','prabowo','bossman','gusbaha'] # The topics of tweets to receive
host   = "0.0.0.0"       # set untuk server pada socketStream

consumer_key = 'JEj5tRSA9JWjWV6imMOrUIVWV'
consumer_secret = '7MEa00KHpUbjxble8pdlV74qPbvW2OHqLtjt45QQraJaAzRmAh'
access_token = '935208713551364097-W9Oy0IS2M1dRUQS5MZ6Dnz18BkHUP80'
access_secret = 'jCANa7K7werTP2X1mnLlcRBFDHAJt9TZSCbC77FSNCj50'

## Define Class Listener
# 1. Listening Stream via StreamListener
# 2. Create function (init) untuk inisialisasi socket
# 3. Data berupa tweet dengan format json yang berasal dari proses Stream
# 4. File json tersebut pada field status "text" bersisi aktual tweet
# 5. Lalu aktual tweet diekstrak dan dikirim ke client socket
# 6. Dilengkapi error handling dlm bentuk catch dan throw error

class Listener(StreamListener):

    def __init__(self, csocket, api):
        self.client_socket = csocket
        self.api = api
 
    def on_status(self, status):
        """ handles the incoming tweets by first filtering and then forward to
        socket.

        inputs:
        -------
        - status: the in-coming tweet object itself with many attributes all 
            stored in JSON format. For more attributes of tweet object, -> 
            <https://developer.twitter.com/en/docs/tweets/data-dictionary/
            overview/tweet-object>
        """
        # discards the tweets that are either truncated or not in english
        # if status.truncated or status.lang != "en": return

        # reads the text attribute of status object
        # forward to the client socket provided
        try:
            if status.text != '':
                tmsg = status.text.encode('utf-8')    # encode
                self.client_socket.send(tmsg)
            return True
        # if an exception occurred between connection with spark like broken
        # pipe, stop pulling tweets
        except BaseException as e:
            print("Error on_status: %s" % str(e))
            return False

        return True
 
    def on_error(self, status):
        """ if there is an error occured between connection with tweepy, stop
        pulling tweets.
        """
        print(status)
        return False


def sendData(c_socket=None):
    # set up api and authorization for receiving tweets
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    api = API(auth)
 
    print("sending tweets...")
    
    # receive tweets on assigned tracks, 
    # filter them by assigned conditions and send them to port
    twitter_stream = Stream(auth, Listener(c_socket, api))
    # twitter_stream.filter(lang=["en"],track=tracks)
    twitter_stream.filter(track=tracks,languages=["id","en"])
    
    
## Create Socket
# 1. Socket dibuat menggunakan socket function
# 2. Host socket di local instance (VM) di AWS dengan IP 0.0.0.0 dimana jupyterlab running pada port tertentu
# 3. Port rekomendasi yang digunakan adalah 8501
# 4. Lalu gunakan IP host dan port tersebut untuk socket

if __name__ == "__main__":

    s = socket.socket()     # initial a socket object
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    
    # set 8501 ini sesuai dengan open port yang saya set pada file 
    # docker-compose.yml pada bagian jupyterlab, Andapun bisa mengubahnya
    # ref: https://raw.githubusercontent.com/imamcs19/edubig-spark-standalone-cluster-and-support-streamlit-port-on-docker/master/docker-compose.yml
    # 
    s.bind((host, 8501))    # bind to the port connected with Spark
 
    print("Listening on port: 8501")
 
    s.listen(5)             # Now wait for client connection.
    c, addr = s.accept()    # Establish connection with client.
 
    print("Received request from: " + str(addr))
 
    sendData(c_socket=c)
